In [1]:
import pandas as pd
import os

from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline
from pyspark.sql import functions as fn, Row
from pyspark import sql

import matplotlib.pyplot as plt
import pandas as pd

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

from datetime import datetime
from pyspark.sql.functions import col,udf
from pyspark.sql.types import DateType
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)



In [8]:
raw_weather_holiday_merged = pd.read_excel("/Users/apsharma/IST718 Dropbox/IST 718 Project/Weather Data/2015_weather_holiday_merged.xlsx")
raw_weather_holiday_merged["pickup_date"] = pd.to_datetime(raw_weather_holiday_merged['pickup_date'], format="%d.%m.%y", errors='coerce')
raw_weather_holiday_merged.head()

,pickup_date,avg_temp_C,Rain,Fog,Snow,Holiday
0,2015-01-01,1,0,0,0,1
1,2015-01-02,4,0,0,0,0
2,2015-01-03,3,1,0,1,0
3,2015-01-04,9,1,0,0,0
4,2015-01-05,2,0,0,0,0


In [9]:
raw_weather_holiday_merged = spark.createDataFrame(raw_weather_holiday_merged)
raw_weather_holiday_merged = raw_weather_holiday_merged.withColumn("date_only", fn.to_date(fn.col("pickup_date")))
raw_weather_holiday_merged.show()

+-------------------+----------+----+---+----+-------+----------+
|        pickup_date|avg_temp_C|Rain|Fog|Snow|Holiday| date_only|
+-------------------+----------+----+---+----+-------+----------+
|2015-01-01 00:00:00|         1|   0|  0|   0|      1|2015-01-01|
|2015-01-02 00:00:00|         4|   0|  0|   0|      0|2015-01-02|
|2015-01-03 00:00:00|         3|   1|  0|   1|      0|2015-01-03|
|2015-01-04 00:00:00|         9|   1|  0|   0|      0|2015-01-04|
|2015-01-05 00:00:00|         2|   0|  0|   0|      0|2015-01-05|
|2015-01-06 00:00:00|        -6|   0|  0|   1|      0|2015-01-06|
|2015-01-07 00:00:00|        -9|   0|  0|   0|      0|2015-01-07|
|2015-01-08 00:00:00|        -9|   0|  0|   0|      0|2015-01-08|
|2015-01-09 00:00:00|        -3|   0|  0|   1|      0|2015-01-09|
|2015-01-10 00:00:00|        -7|   0|  0|   0|      0|2015-01-10|
|2015-01-11 00:00:00|        -2|   0|  0|   0|      0|2015-01-11|
|2015-01-12 00:00:00|         3|   1|  0|   0|      0|2015-01-12|
|2015-01-1

In [10]:
weather_holiday_data_clean = raw_weather_holiday_merged

In [11]:
raw_Jan_Data1 = pd.read_csv("/Users/apsharma/IST718 Dropbox/IST 718 Project/Taxi Data/2015-01_100k.csv")
raw_Feb_Data1 = pd.read_csv("/Users/apsharma/IST718 Dropbox/IST 718 Project/Taxi Data/2015-02_100k.csv")
raw_Mar_Data1 = pd.read_csv("/Users/apsharma/IST718 Dropbox/IST 718 Project/Taxi Data/2015-03_100k.csv")
raw_Apr_Data1 = pd.read_csv("/Users/apsharma/IST718 Dropbox/IST 718 Project/Taxi Data/2015-04_100k.csv")
raw_May_Data1 = pd.read_csv("/Users/apsharma/IST718 Dropbox/IST 718 Project/Taxi Data/2015-05_100k.csv")
raw_Jun_Data1 = pd.read_csv("/Users/apsharma/IST718 Dropbox/IST 718 Project/Taxi Data/2015-06_100k.csv")
raw_Jul_Data1 = pd.read_csv("/Users/apsharma/IST718 Dropbox/IST 718 Project/Taxi Data/2015-07_100k.csv")
raw_Aug_Data1 = pd.read_csv("/Users/apsharma/IST718 Dropbox/IST 718 Project/Taxi Data/2015-08_100k.csv")
raw_Sep_Data1 = pd.read_csv("/Users/apsharma/IST718 Dropbox/IST 718 Project/Taxi Data/2015-09_100k.csv")
raw_Oct_Data1 = pd.read_csv("/Users/apsharma/IST718 Dropbox/IST 718 Project/Taxi Data/2015-10_100k.csv")
raw_Nov_Data1 = pd.read_csv("/Users/apsharma/IST718 Dropbox/IST 718 Project/Taxi Data/2015-11_100k.csv")
raw_Dec_Data1 = pd.read_csv("/Users/apsharma/IST718 Dropbox/IST 718 Project/Taxi Data/2015-12_100k.csv")


In [12]:
frames = [raw_Jan_Data1,raw_Feb_Data1,raw_Mar_Data1,raw_Apr_Data1,raw_May_Data1,raw_Jun_Data1,raw_Jul_Data1,
          raw_Aug_Data1,raw_Sep_Data1,raw_Oct_Data1,raw_Nov_Data1,raw_Dec_Data1]

In [13]:
Full_DataPD = pd.concat(frames)

In [14]:
Full_DataPD1=Full_DataPD
Full_DataPD1 = spark.createDataFrame(Full_DataPD1)

In [15]:
Full_DataPD2 = Full_DataPD1.withColumn("date_only", fn.to_date(fn.col("pickup_datetime")))
Full_DataPD2.limit(20).show(3)

+--------+-------------------+-------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+-----------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+----------+--------------+--------------------+-----------+---------------+--------------------+----------+
|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RateCodeID|store_and_fwd_flag| dropoff_longitude| dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|pickup_zip|pickup_borough| pickup_neighborhood|dropoff_zip|dropoff_borough|dropoff_neighborhood| date_only|
+--------+-------------------+-------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+-----------------+------------+-----------+-----+--

In [16]:
Merged_FULL_data = weather_holiday_data_clean.join(Full_DataPD2, ['date_only'], how='full') #execute this when needed
## Important --  Merged_FULL_data = Full_DataPD2.join(weather_holiday_data_clean, ['date_only'], how='left') #not working

In [18]:
training_df, validation_df, testing_df = Merged_FULL_data.randomSplit([0.6, 0.3, 0.1], seed=0)

In [19]:
##training_df.write.csv('Training_Data_one.csv') #Saves in Parts !
training_df.toPandas().to_csv("Training_Data_Final.csv", header=True)

In [20]:
##validation_df.write.csv('Validation_Data_one.csv')
validation_df.toPandas().to_csv("Validation_Data_Final.csv", header=True)

In [21]:
##testing_df.write.csv('Testing_Data_one.csv')
testing_df.toPandas().to_csv("Testing_Data_Final.csv", header=True)

In [22]:
print("# points in Merged_FULL_data: ", Merged_FULL_data.count()) #must be 12 Million rows

# points in Merged_FULL_data:  1200000


In [23]:
print("# points in training: ", training_df.count()) #must be 719469 rows

# points in training:  719469


In [24]:
print("# points in validation: ", validation_df.count()) #must be 360063 rows

# points in validation:  360063


In [25]:
print("# points in testing: ", testing_df.count()) #must be 120468 rows

# points in testing:  120468


In [26]:
Merged_FULL_data.write.csv('Merged_FULL_data.csv')

In [28]:
Merged_FULL_data.coalesce(1).write.option("header", "true").csv("Merged_FULL_data_Final.csv")

In [ ]:
### ----------------------START BELOW !--------------------- ####

In [ ]:
Training_Data = pd.read